In [22]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent, Tool, AgentExecutor, AgentType

# Initialize the model 'Conversational' variant of GPT-4o
llm = ChatOpenAI(temperature=0.9, model_name="gpt-4o")

# Initialize SerpAPI tool using wrapper
serpapi_wrapper = SerpAPIWrapper()

serpapi_tool = Tool(
    name="serpapi",
    description="busca informacion en la web usando una tool -serpapi-.",
    func=serpapi_wrapper.run
)

# Create an agent with SerpAPI tool
agent = initialize_agent(
    tools=[serpapi_tool],
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True
)

# First chain: Agent generates text using SerpAPI
story_prompt = PromptTemplate.from_template(
    "Usa la herramienta de busqueda para buscar un texto de Borges en la web (de 3 a 5 lineas) relacionadas con alguno de sus mas famosos textos."
)
# story_chain = story_prompt | agent | StrOutputParser()
story_chain = story_prompt | agent | (lambda x: x['output']) | StrOutputParser()

# Second chain analyzes the story
analysis_prompt = PromptTemplate.from_template(
    "Analiza en sentimiento de la historia:\n{story}"
)
analysis_chain = analysis_prompt | llm | StrOutputParser()

# Third chain: Rewrite the story with a specified sentiment
rewrite_prompt = PromptTemplate.from_template(
    "Reescribe la siguiente historia con un sentimiento {sentiment}:\n{story}"
)
rewrite_chain = rewrite_prompt | llm | StrOutputParser()

# Combine chains
# story_with_analysis = story_chain | analysis_chain
# Run the combined chain
# story_analysis = story_with_analysis.invoke({"topic": "a rainy day"})
# print("\nAnalysis:", story_analysis)

# Using RunnablePassthrough.assign to preserve original data
enhanced_chain = RunnablePassthrough.assign(
    story=story_chain  # Add 'story' key with generated content
).assign(
    analysis=analysis_chain  # Add 'analysis' key with analysis of the story
).assign(
    rewritten_story=lambda x: rewrite_chain.invoke({"story": x["story"], "sentiment": "humoristico"})
)

# Execute the chain
result = enhanced_chain.invoke({"topic": "El azar"})
print("Story:", result['story'])
print("\nAnalysis:", result['analysis'])
print("\nRewritten Story (Happy):", result['rewritten_story'])
print(result.keys())



Story: Here is a short excerpt related to one of Borges's renowned stories, "El Aleph":

"Cerré los ojos, los abrí. Entonces vi el Aleph. Arribo, ahora, al inefable centro de mi relato; empieza, aquí, mi desesperación de escritor."

This line encapsulates the moment when the narrator perceives the infinite space contained within the Aleph, highlighting Borges's exploration of infinity and the limitless nature of the universe.

Analysis: La oración del extracto de "El Aleph" de Jorge Luis Borges transmite una mezcla de asombro, maravilla y cierta ansiedad o angustia. La sensación predominante es la de lo sublime y lo inefable, ya que el narrador describe el momento en que percibe el Aleph, un punto que contiene todo el infinito del universo. Este descubrimiento es abrumador y desafiante para el narrador, lo que se ve reflejado en la "desesperación de escritor" que menciona. Esto sugiere una lucha interna para captar y expresar en palabras algo que trasciende la comprensión humana. La na